In [6]:
int(1.25)

1

In [7]:
from telebot import TeleBot
from telebot.types import Message
from io import BytesIO
from image_manager import init_board
import db


# Токен вашего бота, полученный от https://t.me/BotFather
TOKEN: str = "ВАШ_ТОКЕН"  # TODO: Подставляем токен

# TODO: сервер бота всё ещё не совсем работоспособен, это нужно исправить

# Создаем экземпляр бота с указанным токеном
bot: TeleBot = TeleBot(TOKEN)


@bot.message_handler(commands=["start"])
def start(message: Message) -> None:
    """
    Обрабатывает команду /start и отправляет приветственное сообщение
    вместе с изображением шахматной доски.

    :param message: Объект сообщения, содержащий информацию о чате.
    """
    bot.send_message(message.chat.id, "Привет!")
    board = init_board()
    image_io = BytesIO()
    board.save(image_io, "PNG")
    image_io.seek(0)
    bot.send_photo(message.chat.id, image_io)


@bot.message_handler(commands=["create_game"])
def create_game(message):
    """
    Создает новую игру для пользователя.

    Args:
        message: Объект сообщения от пользователя, содержащий информацию о команде.
    """
    if not message.from_user:
        return

    # Создаем игру и получаем уникальный идентификатор игры
    game_id = db.create_game(message.from_user.id)
    # Отправляем пользователю сообщение с id созданной игры
    bot.send_message(message.from_user.id, "Вы создали игру с id: " + game_id)


@bot.message_handler(commands=["join_game"])
def join_game(message: Message):
    """
    Позволяет пользователю присоединиться к существующей игре.

    Args:
        message: Объект сообщения от пользователя, содержащий команду и id игры.
    """
    if not (message.from_user and message.text):
        return

    # Разделяем текст сообщения, чтобы получить id игры
    _, game_id = message.text.split(" ")

    # Проверяем, не полна ли игра
    if db.game_full(game_id):
        # Если игра уже началась, отправляем соответствующее сообщение
        bot.send_message(message.chat.id, "Игра уже началась")
        return

    # Если игра не полна, присоединяем пользователя к игре
    db.join_game(message.from_user.id, game_id)
    # Отправляем пользователю сообщение о том, что он присоединился к игре
    bot.send_message(message.chat.id, "Вы присоединились к игре с id: " + message.text)

    # Получаем id белого игрока (например, первого игрока в игре)
    white_user_id = db.get_user_id(game_id, "white")

    # Если белый игрок существует, уведомляем его о новом участнике
    if white_user_id:
        bot.send_message(white_user_id[0], "К игре присоединился игрок " + message.from_user.first_name)


if __name__ == "__main__":
    bot.polling(none_stop=True)